<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/2%20Advanced/100MolecularOrbital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colaboratory上で分子軌道計算を試します。

[Labo-code](https://labo-code.com/python/quantum-chemical-calculation/moleculaorbital/)の記事を参考にしました。感謝!


# 1. 初期化 / 1. Initialization


## 1.1 必要なパッケージのインストール / 1.1 Requirements

Some modules are not pre-installed in the Colaboratory.


In [ ]:
!pip install pyscf
!pip install geometric
!pip install py3Dmol

# 2. 構造最適化 / 2. Structure Optimization



## 2.1 初期データを作る / 2.1 Prepare the initial data

「直角」水分子を準備します。

We prepare a "right-angled" water molecule for the calculation.



In [ ]:
from pyscf import gto
# GTO: Gaussian-type Orbital


# 水分子の定義 (座標の単位はÅ、すごくてきとう。)
# Définition d'une molécule d'eau
mol = gto.Mole()
mol.atom = """
O 0.0 0.0 0.0
H 1.0 0.0 0.0
H 0.0 1.0 0.0
"""
mol.basis='6-31G(d)'
mol.build()

In [ ]:
# とりあえず、そのまま表示してみる
# Pour l'instant, affichons-le tel quel.
import py3Dmol
from IPython.display import display, HTML


def show_mol(mol, viewer):
    # 分子のサイズを取得
    # Obtain the size of a molecule
    # Obtenir la taille d'une molécule
    coords = mol.atom_coords()
    x_min, y_min, z_min = coords.min(axis=0) - 2.0
    x_max, y_max, z_max = coords.max(axis=0) + 2.0


    # 分子構造をviewerに追加
    # Add the molecular structure to the viewer
    # Ajouter la structure moléculaire à la viewer
    xyz = mol.tostring(format="xyz")
    viewer.addModel(xyz, "xyz")
    viewer.setStyle({"stick": {}, "sphere": {"scale": 0.3}})


# 3D表示の準備
# Preparation de l'affichage en 3D
viewer = py3Dmol.view(width=800, height=600)
show_mol(mol, viewer)
viewer.zoomTo()
viewer.show()


## 2.2 構造最適化 / Structure Optimization

与えられた原子核の初期配置に対し、電子を配置してエネルギーができるだけ低くなるように電子と原子核の配置を最適にします。これを構造最適化と呼びます。

For a given initial configuration of nuclei, the arrangement of electrons and nuclei is optimized so that the electrons are placed with the lowest possible energy. This is known as structural optimization.

Pour une configuration initiale donnée des noyaux, l'arrangement des électrons et des noyaux est optimisé de manière à ce que les électrons soient placés de façon à ce que l'énergie soit la plus faible possible. C'est ce qu'on appelle l'optimisation structurelle.

In [ ]:
from pyscf.geomopt import geometric_solver
from pyscf import dft, scf

# DFT計算のセットアップ
# A setup for the DFT calculations

# mf = dft.RKS(mol)

# SCF計算のセットアップ
# A setup for the SCF calculations
mf = scf.RHF(mol)

# 構造最適化
# Structure optimization
mol_eq = geometric_solver.optimize(mf, maxsteps=100)


In [ ]:
view = py3Dmol.view(width=800, height=600)
show_mol(mol_eq, view)
view.zoomTo()
view.show()


## 2.3 電子配置 / 2.3 Electronic arrangements

電子を、エネルギーの低い軌道から順に入れていきます。酸素と水素で全部で10個の電子があるので、5つの軌道が埋まります。

エネルギーの単位をhartreeからeVに換算して表示します。

Electrons are placed first in the least energetic orbitals. There are 10 electrons in total in oxygen and hydrogen, so five orbitals are filled.

Convert energy units from hartree to eV.

Les électrons sont placés en premier dans les orbitales les moins énergétiques. Il y a 10 électrons au total dans l'oxygène et l'hydrogène, donc cinq orbitales sont remplies.

Convertissez les unités d'énergie de hartree en eV.

In [ ]:
# 軌道情報の(再)計算
# (Re-)calculation of the orbitals
# (Re-)calcul des orbitales
mf.kernel()

# エネルギーをeVに変換
# Convert the energy in eV
# Convertir l'energie en eV
hartree_to_ev = 27.2114

mf.mo_occ, mf.mo_energy * hartree_to_ev


電子が入っている、もっともエネルギーの高い(最も外側にある)軌道のことをHOMO (Highest Occupied Molecular Orbital; 最高被占分子軌道、その一つ上の軌道のことをLUMO (Lowest Unoccupied Molecular Orbital; 最低空分子軌道)と呼びます。一番外側の軌道は、分子の形や大きさや反応性を決定づけるので、化学ではこれらの軌道に特に注目します。

水の場合、HOMOは5番目、LUMOは6番目の軌道です。

Pythonでは、数字を0から数えはじめるので、これらはそれぞれ4番目、5番目に相当します。

The orbital of the highest energy (outermost) containing an electron is called HOMO (Highest Occupied Molecular Orbital), and the empty orbital above it is called LUMO (Lowest Unoccupied Molecular Orbital). These outer orbitals are of particular interest in chemistry, as they determine the shape, size and reactivity of molecules.

In the case of water, HOMO is the fifth orbital and LUMO the sixth.

In Python, we start counting numbers from 0, so these orbitals correspond to the fourth and fifth orbitals respectively.

L'orbitale la plus énergétique (la plus externe) contenant un électron est appelée HOMO (Highest Occupied Molecular Orbital), et l'orbitale vide située au-dessus est appelée LUMO (Lowest Unoccupied Molecular Orbital). Ces orbitales externes sont particulièrement intéressantes en chimie, car elles déterminent la forme, la taille et la réactivité des molécules.

Dans le cas de l'eau, HOMO est la cinquième orbitale et LUMO la sixième.

En Python, nous commençons à compter les nombres à partir de 0. Ces orbitales correspondent donc respectivement à la quatrième et à la cinquième orbitale.

In [ ]:
import numpy as np

# HOMOとLUMOのインデックス
lumo_index = np.argwhere(mf.mo_occ == 0)[0][0]
homo_index = lumo_index - 1
lumo_index, homo_index

## 2.4. 軌道の可視化 / Visualization of the orbitals

途中で、軌道の形をcube形式のファイルに書きだし、それを読みこんでいます。

During the process, the trajectory shape is exported to a file in CUBE format, which is then read.

Au cours du processus, la forme de la trajectoire est exportée vers un fichier au format CUBE, qui est ensuite lu.

In [ ]:
from pyscf import tools
import py3Dmol
from IPython.display import display, HTML
import ipywidgets as widgets

# 指定した軌道のCUBEファイルを生成する関数
def generate_cube_file(orbital_index, filename):
    tools.cubegen.orbital(mol, filename, mf.mo_coeff[:, orbital_index])

# 可視化する軌道のリストを作成
orbitals = {
    "LUMO+2": lumo_index + 2,
    "LUMO+1": lumo_index + 1,
    "LUMO": lumo_index,
    "HOMO": homo_index,
    "HOMO-1": homo_index - 1,
    "HOMO-2": homo_index - 2,
    "HOMO-3": homo_index - 3,
    "HOMO-4": homo_index - 4
}

# エネルギーをeVに変換
hartree_to_ev = 27.2114
orbital_energies = {name: mf.mo_energy[idx] * hartree_to_ev for name, idx in orbitals.items()}

# 可視化関数
def show_orbital(orbitals, orbital_name, view):
    orbital_index = orbitals[orbital_name]
    cube_filename = f'{orbital_name}.cube'

    generate_cube_file(orbital_index, cube_filename)

    with open(cube_filename) as f:
        cube = f.read()
    view.addVolumetricData(cube, 'cube', {'isoval': 0.02, 'color': 'red', 'opacity': 0.75})
    view.addVolumetricData(cube, 'cube', {'isoval': -0.02, 'color': 'blue', 'opacity': 0.75})

    # エネルギーのラベルを追加
    orbital_energy = orbital_energies[orbital_name]
    view.addLabel(f'{orbital_name}: {orbital_energy:.6f} eV', {'fontSize': 12, 'fontColor': 'black', 'backgroundColor': 'white', 'backgroundOpacity': 0.8, 'showBackground': True, 'position': {'x': 0, 'y': 0, 'z': 0}})


Google Colabの簡易フォーム機能を使い、メニューで軌道を選べるようにしました。

Google Colab's form feature was used to enable trajectory selection from the menu.

La fonction de formulaire de Google Colab a été utilisée pour permettre la sélection de la trajectoire à partir du menu.

In [ ]:
# @title 軌道の可視化 / Visualization of the orbitals
orbital_name = "HOMO-1" # @param ["LUMO+2", "LUMO+1", "LUMO", "HOMO", "HOMO-1", "HOMO-2", "HOMO-3", "HOMO-4"]
# @

view = py3Dmol.view(width=800, height=600)
show_mol(mol_eq, view)
show_orbital(orbitals, orbital_name, view)
view.zoomTo()
view.show()


# 3. 相互作用の計算 / Interactions

水とCO2の相互作用エネルギーを計算してみます。

Calculate the interaction energy between water and CO2.

Calculez l'énergie d'interaction entre l'eau et le CO2.




## 3.1 CO2を作る / Make a CO2 molecule / Faire une molecule de CO2


In [ ]:
from pyscf import gto, scf
from pyscf.geomopt import geometric_solver

# --- CO2 ---
co2 = gto.Mole()
co2.atom = [
    ['C', (0.0, 0.0, 0.0)],
    ['O', (1.2, 0.0, 0.0)],
    ['O', (-1.2, 0.0, 0.0)],
]
co2.basis = '6-31g*' # Basis set
co2.build()

mf_co2 = scf.RHF(co2).run()
co2_opt = geometric_solver.optimize(mf_co2)

co2_opt.atom_coords()

## 3.2 ２分子複合体を作る / Prepare a molecular complex

2分子が近くにある配置`combined`を作ります。

Make a molecular configuration in "combined".

In [ ]:
# 最適化されたCO2の座標と原子名を取得
# Obtain the coordinates and names of the atoms in the optimized structure of CO2
atom_names_co2 = [atom[0] for atom in co2_opt._atom]
atom_coords_co2 = co2_opt.atom_coords()

# 水も同様
# And water
# Et de l'eau
atom_names_h2o = [atom[0] for atom in mol_eq._atom]
atom_coords_h2o = mol_eq.atom_coords()

# 水をx方向に8 bohrずらす。
# Displace the water molecule
atom_coords_co2[:, 0] += 8

atom_names_h2o, atom_coords_h2o

In [ ]:
# 複合系の原子リストと座標を作成
# H2OとCO2の原子情報を結合
# Combine the atomic information
combined_atom = []
for i in range(3):
    combined_atom.append([atom_names_h2o[i], *atom_coords_h2o[i]])
for i in range(3):
    combined_atom.append([atom_names_co2[i], *atom_coords_co2[i]])

combined_atom

## 3.3 分子配置を見る / Visualize the molecular configurarion

In [ ]:
mol_complex = gto.M(atom=combined_atom, basis='6-31g*', unit='bohr')

view = py3Dmol.view(width=800, height=600)
show_mol(mol_complex, view)
view.zoomTo()
view.show()


## 3.4 複合体のエネルギー計算 / Energy calculation of the molecular complex

ここでは、原子が動いてほしくないので、構造最適化は行わず、エネルギー計算だけを行います。

We do not want the molecule to move, so we skip the structure optimization.

In [ ]:
# 複合系のエネルギー計算
# Energy calculation of the complex
mf_complex = scf.RHF(mol_complex).run()
# and obtain
e_complex = mf_complex.e_tot
e_complex # in hartree

## 3.5 会合によるエネルギー収支 / Energy balance by combination

In [ ]:
e_h2o = mf.e_tot
e_h2o # in hartree

In [ ]:
e_co2 = mf_co2.e_tot
e_co2 # in hartree

In [ ]:
(e_complex - (e_h2o+e_co2)) * 2625.5 # in kJ/mol

1 hartreeは2625 kJ/mol、かなり巨大な値です。電子状態計算でこんな大きなエネルギーがでてくるのは、原子核とすべての電子の間のクーロンエネルギーを計算するせいです。これに対し、相互作用を計算する場合には、分子が近付いたことによるエネルギー利得(差分)しか扱わないので、ずいぶん桁が小さくなります。したがって、電子状態計算をかなり精密に行わないと、相互作用の値は信用できなくなってしまいます。



# 4. 分子の形を組みたてる / Build a molecule

初期配置がいいかげんでもpySCFが最適化してくれます。とはいえ、あまりにも雑な構造を与えてしまうと、収束しないで計算が終わってしまうようです。また、水分子の初期配置を一直線にした場合にも、対称性を崩せず、V字型の最適配置にたどりつけないようです。

分子の形をそれなりの精度で組み立てるには、[molcalc](https://molcalc.org/)が便利です。MolCalcに組みこまれているJSMolの機能で、分子構造の情報を出力できます。(JSMol→Show→Extract MOL data)

ただし、MolCalcが扱うのは分子=原子+結合であるのに対し、pySCFの扱う電子状態計算には原子座標しか要らないので、データを変換する必要があります。ここでは、MOLファイル形式をじっくり睨み、自分で抽出を試みます。

Even if the initial placement is not appropriate, pySCF will optimize it. But if too messy a structure is given, the calculation seems to end without convergence. Also, if the initial arrangement of water molecules is aligned, it seems that the symmetry cannot be broken and the optimal V-shaped arrangement cannot be reached.

To assemble molecular shapes with reasonable accuracy, [molcalc](https://molcalc.org/) is useful, and the JSMol function incorporated in MolCalc can output molecular structure information. (JSMol→Show→Extract MOL data)

However, MolCalc handles molecules = atoms + bonds, whereas pySCF handles electronic structure calculations, which only require atomic coordinates, so the data must be converted. Here, we will take a closer look at the MOL file format and attempt to extract the data by hand..



In [ ]:
methane_mol="""string
__Jmol-14_06032519493D 1   1.00000     0.00000     0
Jmol version 14.29.29  2018-11-28 19:07 EXTRACT: ({0:4})
  5  4  0  0  0  0            999 V2000
   -0.0002    0.0000   -0.0001 C   0  0  0  0  0  0
   -0.4377    0.4984   -0.8687 H   0  0  0  0  0  0
   -0.2313    0.5713    0.9027 H   0  0  0  0  0  0
   -0.4152   -1.0071    0.0898 H   0  0  0  0  0  0
    1.0843   -0.0626   -0.1238 H   0  0  0  0  0  0
  1  2  1  0  0  0
  1  3  1  0  0  0
  1  4  1  0  0  0
  1  5  1  0  0  0
M  END
""".splitlines()

# 必要な情報は5〜9行目にあるらしい。
# The lines 5..9 are required.
methane = []
for line in methane_mol[4:9]:
    cols = line.split()
    methane.append([cols[3], float(cols[0]), float(cols[1]), float(cols[2])])

methane

In [ ]:
# combinedの時と同じ方法でとりあえず組みたてる。
# Build the methane molecule in the same way as "combined".
mol_methane = gto.M(atom=methane, basis='6-31g*', unit='angstrom')

view = py3Dmol.view(width=800, height=600)
show_mol(mol_methane, view)
view.zoomTo()
view.show()


# 5. Practice

1. MolCalcを使ってベンゼンのような分子$\mathrm {C_6H_6}$を作り、PySCFで構造最適化して、HOMOを表示して下さい。
1. Basis setをSTO-3G、3-21G、6-31G、6-31G(d)と変えた場合に、エネルギーがどう変わるかを計算し、なぜ変わるかを説明して下さい。
1. MolCalc等を使い、自分の好きな分子を組み立て、その構造を最適化しなさい。
---
1. Use MolCalc to make a benzene-like molecule $\mathrm {C_6H_6}$, optimize its structure with PySCF, and display the HOMO.
1. Calculate how the energies change when you change the Basis set to STO-3G, 3-21G, 6-31G, 6-31G(d) and explain why.
1. Using MolCalc or other software, assemble a molecule of your choice and optimize its structure.



